From HW 3: 
Markov chain that computes p(beat_length | previous_beat_length, beat_position)

In [1]:
import numpy as np
import random
from glob import glob
from collections import defaultdict
from numpy.random import choice
from symusic import Score
from miditok import REMI, TokenizerConfig
from midiutil import MIDIFile
import os
from IPython.display import Audio, display

In [2]:
# midi file set up
midi_files = glob('../../data/*.mid')
tokenizer = REMI.from_pretrained("tokenizer.json")

duration2length = {
    '0.1.8': 1,   # 32nd note (0.125 beat)
    '0.2.8': 2,   # 16th note (0.25 beat)
    '0.3.8': 3,   # Dotted 16th (0.375 beat)
    '0.4.8': 4,   # 8th note (0.5 beat)
    '0.5.8': 5,   # Equivalent to 0.625 beats (5/8 of a quarter note)
    '0.6.8': 6,   # Dotted 8th (0.75 beat)
    '0.7.8': 7,   # Approx. 0.875 beats (7/8 of a quarter note)
    '1.0.8': 8,   # Quarter note (1 beat)
    '1.2.8': 10,  # Quarter + 16th (1.25 beat)
    '1.4.8': 12,  # Dotted quarter (1.5 beat)
    '2.0.8': 16,  # Half note (2 beats)
    '3.0.8': 24,  # Dotted half (3 beats)
    '4.0.4': 32,  # Whole note (4 beats)
    '0.1.4': 2,    # 16th note (0.25 beat)
    '0.2.4': 4,    # 8th note (0.5 beat)
    '0.3.4': 6,    # Dotted 8th (0.75 beat)
    '0.4.4': 8,    # Quarter note (1 beat)
    '0.5.4': 10,   # Approx. 1.25 beats
    '0.6.4': 12,   # Approx. 1.5 beats
    '0.7.4': 14,   # Approx. 1.75 beats
    '1.0.4': 16,   # Half note (2 beats)
    '1.2.4': 20,   # Half + 16th (2.5 beats)
    '1.4.4': 24,   # Dotted half (3 beats)
}

/opt/anaconda3/envs/audio-proj/lib/python3.8/site-packages/miditok/tokenizations/remi.py:77: UserWarning: Attribute controls are not compatible with 'config.one_token_stream_for_programs' and multi-vocabulary tokenizers. Disabling them from the config.
  super().__init__(tokenizer_config, params)


Some Code from HW 3 answers

In [3]:
### EXTRACTION ###

def note_extraction(midi_file):
    # Q1a: Your code goes here
    note_events = []
    midi = Score(midi_file)
    rh_track = midi.tracks[0] 
    rh_score = Score()
    rh_score.tracks.append(rh_track)
    tokens = tokenizer(rh_score).tokens 
    for token in tokens:
        if 'Pitch' in token:
            note = int(token.split('_')[1])
            note_events.append(note)
    return note_events

def note_frequency(midi_files):
    note_counts = defaultdict(int)
    for midi_file in midi_files:
        note_events = note_extraction(midi_file)
        for note in note_events:
            note_counts[note] += 1
    return note_counts

def beat_extraction(midi_file):
    midi = Score(midi_file)
    rh_track = midi.tracks[0]
    rh_score = Score()
    rh_score.tracks.append(rh_track)
    tokens = tokenizer(rh_score).tokens

    beats = []

    i = 0
    while i < len(tokens) - 4:
        if tokens[i].startswith('Position') and tokens[i+2].startswith('Pitch') and tokens[i+4].startswith('Duration'):
            position_token = tokens[i]
            duration_token = tokens[i+4]

            beat_position = int(position_token.split('_')[1])
            duration_str = duration_token.split('_')[1]
            beat_length = duration2length.get(duration_str, None)

            if beat_length is not None:
                beats.append((beat_position, beat_length))

            i += 5  # Move past the entire note event
        else:
            i += 1  # Skip malformed group

    return beats


### PROBABILITIES ### 
def note_unigram_probability(midi_files):
    note_counts = note_frequency(midi_files)
    
    # Q2: Your code goes here
    unigramProbabilities = {}
    counts = sum(list(note_counts.values()))
    for n in note_counts:
        unigramProbabilities[n] = note_counts[n] / counts
    return unigramProbabilities

def note_bigram_probability(midi_files):
    # Q3a: Your code goes here
    bigrams = defaultdict(int)
    
    for file in midi_files:
        note_events = note_extraction(file)
        for (note1, note2) in zip(note_events[:-1], note_events[1:]):
            bigrams[(note1, note2)] += 1
            
    bigramTransitions = defaultdict(list)
    bigramTransitionProbabilities = defaultdict(list)

    for b1,b2 in bigrams:
        bigramTransitions[b1].append(b2)
        bigramTransitionProbabilities[b1].append(bigrams[(b1,b2)])
        
    for k in bigramTransitionProbabilities:
        Z = sum(bigramTransitionProbabilities[k])
        bigramTransitionProbabilities[k] = [x / Z for x in bigramTransitionProbabilities[k]]
        
    return bigramTransitions, bigramTransitionProbabilities

def note_trigram_probability(midi_files):
    # Q5a: Your code goes here
    trigrams = defaultdict(int)
    for file in midi_files:
        note_events = note_extraction(file)
        for (note1, note2, note3) in zip(note_events[:-2], note_events[1:-1], note_events[2:]):
            trigrams[(note1, note2, note3)] += 1
            
    trigramTransitions = defaultdict(list)
    trigramTransitionProbabilities = defaultdict(list)

    for t1,t2,t3 in trigrams:
        trigramTransitions[(t1,t2)].append(t3)
        trigramTransitionProbabilities[(t1,t2)].append(trigrams[(t1,t2,t3)])
        
    for k in trigramTransitionProbabilities:
        Z = sum(trigramTransitionProbabilities[k])
        trigramTransitionProbabilities[k] = [x / Z for x in trigramTransitionProbabilities[k]]
        
    return trigramTransitions, trigramTransitionProbabilities


def beat_pos_bigram_probability(midi_files):
    # Q8a: Your code goes here
    bigramBeatPos = defaultdict(int)
    for file in midi_files:
        beats = beat_extraction(file)
        for beat in beats:
            bigramBeatPos[(beat[0], beat[1])] += 1
            
    bigramBeatPosTransitions = defaultdict(list)
    bigramBeatPosTransitionProbabilities = defaultdict(list)

    for b1,b2 in bigramBeatPos:
        bigramBeatPosTransitions[b1].append(b2)
        bigramBeatPosTransitionProbabilities[b1].append(bigramBeatPos[(b1,b2)])
        
    for k in bigramBeatPosTransitionProbabilities:
        Z = sum(bigramBeatPosTransitionProbabilities[k])
        bigramBeatPosTransitionProbabilities[k] = [x / Z for x in bigramBeatPosTransitionProbabilities[k]]
        
    return bigramBeatPosTransitions, bigramBeatPosTransitionProbabilities


def beat_trigram_probability(midi_files):
    # Q9a: Your code goes here
    trigramBeat = defaultdict(int)
    for file in midi_files:
        beats = beat_extraction(file)
        for (beat1, beat2) in zip(beats[:-1], beats[1:]):
            trigramBeat[(beat1[1], beat2[0], beat2[1])] += 1
            
    trigramBeatTransitions = defaultdict(list)
    trigramBeatTransitionProbabilities = defaultdict(list)

    for t1,t2,t3 in trigramBeat:
        trigramBeatTransitions[(t1,t2)].append(t3)
        trigramBeatTransitionProbabilities[(t1,t2)].append(trigramBeat[(t1,t2,t3)])
        
    for k in trigramBeatTransitionProbabilities:
        Z = sum(trigramBeatTransitionProbabilities[k])
        trigramBeatTransitionProbabilities[k] = [x / Z for x in trigramBeatTransitionProbabilities[k]]
        
    return trigramBeatTransitions, trigramBeatTransitionProbabilities


### GENERATE MUSIC ###

def music_generate(length, name):
    # sample notes
    unigramProbabilities = note_unigram_probability(midi_files)
    bigramTransitions, bigramTransitionProbabilities = note_bigram_probability(midi_files)
    trigramTransitions, trigramTransitionProbabilities = note_trigram_probability(midi_files)
    
    # Your code goes here ...
    first_note = choice(list(unigramProbabilities.keys()), 1, p=list(unigramProbabilities.values())).item()
    second_note = choice(bigramTransitions[first_note], 1, p=bigramTransitionProbabilities[first_note]).item()
    sampled_notes = [first_note, second_note]
    while len(sampled_notes) < length:
        next_note = choice(trigramTransitions[(sampled_notes[-2], sampled_notes[-1])], 1, 
                            p=trigramTransitionProbabilities[(sampled_notes[-2], sampled_notes[-1])])
        sampled_notes.append(next_note.item())
    
    # sample beats
    bigramBeatPosTransitions, bigramBeatPosTransitionProbabilities = beat_pos_bigram_probability(midi_files)
    first_beat = choice(bigramBeatPosTransitions[0], 1, p=bigramBeatPosTransitionProbabilities[0]).item()
    sampled_beats = [(0, first_beat)]
    while len(sampled_beats) < length:
        beat_position = sum(sampled_beats[-1]) % 32
        beat_length = choice(bigramBeatPosTransitions[beat_position], 1, 
                        p=bigramBeatPosTransitionProbabilities[beat_position]).item()
        sampled_beats.append((beat_position, beat_length))
    sampled_beats = [beat[1] / 8 for beat in sampled_beats]
    
    # save the generated music as a midi file
    midi = MIDIFile(1)
    track = 0
    time = 0
    tempo = 60 #120
    midi.addTempo(track, time, tempo)
    
    current_time = 0
    for pitch, duration in zip(sampled_notes, sampled_beats):
        midi.addNote(track, 0, pitch, current_time, duration, 100)
        current_time += duration

    # Ensure directory exists
    os.makedirs("markov_model", exist_ok=True)

    # Then write the MIDI file
    with open("markov_model/{}.mid".format(name), "wb") as f:
        midi.writeFile(f)


In [5]:
# generate some examples
max_len = 100

for i in range(1,51):
    music_generate(max_len, 'markov_{}'.format(str(i)))